# Thoughts on pip integration in FreeCAD

find a way to to integrate pip-support in freecad which works with all builds. Mostly this means finding a compromise which works for bundles and for system versions of FreeCAD.

1. app / gui
providing a simple way to install packages from the command line is the first step. Once we figure out a way that works for all builds we can extend this to gui-functionality.

2. api design
make the api-design as simple as possible and similar to pip command-line-tool

3. --user flag
As we cannot and don't want to install to any root directories we are forced to use the --user flag. This will install to a local directory.

4. support different directories
This will give the opportunity to select between different dependency-trees, try out new things,.. without the need to reinstall freecad. (Similar to what environments provide for systems)

5. constraint file
The constraint file provides a method to install stuff in a more secure (stable) way. In bundles and system-installation of freecad, there are some packages already preinstalled. To not mess with these packages and introduce incompabilities we should constraint this packages to there installed versions. This way pip will not update any of the constraint packages. For example this file can be created everytime the freecad.pip package is imported. All packages which are not part of the user site package or a custom site package are listed in this constraint file.
It would be also nice to add additional constraints to this file. But currently I am not sure how to acchieve this.



In [3]:
import os
import copy
import tempfile
import subprocess as subp

print_msg = print
print_err = print

def process(*args):
    proc = subp.Popen(args, stdout=subp.PIPE, stderr=subp.PIPE)
    out, err = proc.communicate()
    if err:
        raise RuntimeError(err.decode("utf8"))
    return out.decode("utf8")

class _pip(object):
    def __init__(self):
        self.constraint_file = tempfile.mktemp(prefix="constraints", suffix=".txt")
        self.freeze()

    def _c_option(self):
        """
        internal function, returns the option to constraint packages
        """
        return "-c{}".format(self.constraint_file)

    @staticmethod
    def _convert_pkgs_list(text):
        if text:
            return [i.split()[:2] for i in text.split("\n")[2:-1]]
        else:
            return []

    def install(self, pkg_name):
        print_msg(process("pip", "install", pkg_name, "--user", self._c_option()))
    
    def install_develop(self, fp):
        print_msg(process("pip", "install", "-e", fp, "--user", self._c_option()))

    def uninstall(self, pkg_name):
        if pkg_name not in [i[0] for i in self.list_user()]:
            print_err("pkg is not a user-package")
        else:
            print_msg(process("pip", "uninstall", pkg_name, "-y"))
    
    def list(self):
        """
        lists all packages
        """
        packages = process("pip", "list")
        return self._convert_pkgs_list(packages)
    
    def list_user(self):
        """
        lists all user packages
        """
        packages = process("pip", "list", "--user")
        return self._convert_pkgs_list(packages)

    def list_editable(self):
        """
        lists all packages
        """
        packages = process("pip", "list", "--editable")
        return self._convert_pkgs_list(packages)
    
    def list_system(self):
        editable = self.list_editable()
        user = self.list_user()
        non_system = editable + user
        return [pkg for pkg in self.list() if not pkg in non_system]

    def freeze(self):
        """
        sets all installed packages fixed. This means these packages won't be updated.
        """
        with open(self.constraint_file, "w") as fp:
            for pkg_name, version in self.list_system():
                fp.write("{}=={}\n".format(pkg_name, version))         

    def set_fixed(self, pkg_name, fixed=True):
        """
        sets the package fixed, or release a fixed package if package is fixed and argument fixed is False
        """
        pass

    def select_user_install_dir(self, install_dir):
        """
        Advanced option to sets the user install dir. This allows to use different directories
        for 3rd-party packages. This can be useful if different addons need different
        dependency-versions. This will require a restart of FreeCAD, because sys.path has to be 
        recomputed.
        """
        os.env["PYTHONUSERBASE"] = install_dir

pip = _pip()


In [4]:
pip.install("template-extension")
pip.install("cadquery")

In [5]:
pip.list_user()

[['cadquery', '1.2.0'], ['template-extension', '0.7']]

In [6]:
pip.uninstall("template-extension")
pip.uninstall("cadquery")

Uninstalling template-extension-0.7:
  Successfully uninstalled template-extension-0.7

Uninstalling cadquery-1.2.0:
  Successfully uninstalled cadquery-1.2.0



In [7]:
pip.list_user()

[]

In [8]:
pip.list_editable()

[['freecad.pip', '0.0.1']]

In [9]:
pip.install_develop("/home/k/projects/OpenGlider/")

Obtaining file:///home/k/projects/OpenGlider
  Using cached https://files.pythonhosted.org/packages/e6/4f/2f20a241ff57297109241842423d23887901233a5613e179bf2e0254ec18/pyexcel_ods-0.5.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9d/32/1428fe19eba9e3ebedd00194ecb2c7261d7114aa72c9aed019097b746066/odfpy-1.3.5.tar.gz
  Running setup.py bdist_wheel for texttable: started
  Running setup.py bdist_wheel for texttable: finished with status 'done'
  Stored in directory: /home/k/.cache/pip/wheels/be/6d/c5/53bc7f399edd50882b855679c1e3667e596f21fee9c8fa35db
  Running setup.py bdist_wheel for odfpy: started
  Running setup.py bdist_wheel for odfpy: finished with status 'done'
  Stored in directory: /home/k/.cache/pip/wheels/dd/26/a2/f1fc5deeaca8726a236fc27a346a703caa3b859cb763b294c8
Successfully built texttable odfpy
  Running setup.py develop for OpenGlider



In [10]:
pip.list_user()

[['gmsh-interop', '2017.1'],
 ['lml', '0.0.4'],
 ['odfpy', '1.3.5'],
 ['OpenGlider', '0.1'],
 ['pyexcel', '0.5.9.1'],
 ['pyexcel-io', '0.5.9.1'],
 ['pyexcel-ods', '0.5.3'],
 ['texttable', '1.4.0']]

In [11]:
pip.list_editable()

[['freecad.pip', '0.0.1'], ['OpenGlider', '0.1']]